In [ ]:
import numpy
import matplotlib.pyplot as plt
from scipy.signal.spectral import lombscargle

# Test with Artificial Signal

Start with regularly sampled signal

In [ ]:
xvar = numpy.arange(0,128,1.0)

fft = numpy.zeros(shape=(128),dtype=numpy.complex128)
lmb = numpy.zeros(shape=(64))
# Test averaging
averages = 1000
for i in numpy.arange(averages):

    sine_test = numpy.sin(2*numpy.pi * 0.04 * xvar) + numpy.random.randn(128)/5 +numpy.sin(2*numpy.pi * 0.08 * xvar)

    #plt.plot(xvar,sine_test)
    #plt.show()

    hamm_sine = numpy.hamming(128) * sine_test
    ## FFT
    fft += numpy.fft.fft(hamm_sine)/128
    freqs = numpy.fft.fftfreq(len(fft))
    
    ## Lombscargle
    freqs = numpy.arange(0.0001,1.28,0.02)
    lmb += lombscargle(xvar, hamm_sine,freqs)

    
plt.semilogy(xvar,numpy.square(numpy.abs(fft/averages)))    
plt.semilogy(numpy.arange(0,64,1),numpy.abs(lmb/averages))


## Irregularly Sampled Signal

A single flagged out channel

In [ ]:
randn = numpy.random.randint(0,128)
fft = numpy.zeros(shape=(128),dtype=numpy.complex128)
lmb = numpy.zeros(shape=(64))
averages=1000
for i in numpy.arange(averages):

    sine_test = numpy.sin(2*numpy.pi * 0.04 * xvar) + numpy.random.randn(128)/5+numpy.sin(2*numpy.pi * 0.06 * xvar)
    hamm_sine = numpy.hamming(128) * sine_test
    hamm_sine_test_fft = hamm_sine.copy()
    
    hamm_sine_test_fft[randn] = 0.0
    hamm_sine_test_lmb = hamm_sine.copy()
    hamm_sine_test_lmb = numpy.delete(hamm_sine_test_lmb,randn)
    xvar_lmb = xvar.copy()
    xvar_lmb = numpy.delete(xvar_lmb, randn)
    #plt.plot(xvar,sine_test)
    #plt.show()

    
    ## FFT
    fft += numpy.fft.fft(hamm_sine_test_fft)/128
    freqs = numpy.fft.fftfreq(len(fft))
    
    ## Lombscargle
    freqs = numpy.arange(0.0001,0.64,0.01)
    lmb += lombscargle(xvar_lmb, hamm_sine_test_lmb,freqs)/128

    
plt.semilogy(xvar,numpy.square(numpy.abs(fft/averages)))    
plt.semilogy(numpy.arange(0,64,1),numpy.abs(lmb/averages))